In [8]:
#지역별로 전기차 데이터(csv)와 지역별 전기차 충전소(위치이름, 위도, 경도, 충전속도, 등등)를 가지고 분석하여 지역별로 전기차 구매가 적합한지 아닌지 분석 해본다.
#쾌속인지 완속인지, 에 가중치를 두어서 충전기에 몇개의 전기차가 평균적으로 붙는지 지역별로 계산하여 '지역별 전기차 구매 적합 '지수'' 를 도출해본다.
#도출한 지수를 가지고 충전소를 표시한 지도에 지역에 지수를 추가하여 시각적인 정보를 극대화한다.

# +) 추가적으로 왜 전기차가 폭발적으로 성장하게 되었는지도 분석해본다. (년도별 전기차 판매대수와 년도별 전기차 보조금 비교)


# 1 이상치 300여건의 이유.  - 진행중 ( 문자 받았는데, 확인후 연락주신다고 함. )

# 2 전기차충전소의 늘어난 시기와 이유(정책, 수요증가), - 보조금의 이유일것이라고 추정 (진행중)

# 3 전기차등록대수 뿐 아니라 인구통계도 인구통계와 경제수준을 평가해서   - 추가로 수행 / 아마 등록대수와 비례할것이라고 예상

# 4 전기차 증가에대한 코로나 연관성, - 추가로 수행    /  코로나와 직접적인 연관은 없지만, 코로나로 인한 소비위축으로 소비장려를 위한 보조금을 전기차 비중을 늘려서 간접적인 연관이 있을것으로 예상            

# 5 전기차를 타기에 적합한가 왜 전기차로 넘어가고 있는지 보급률 추이. - 2와 동일

# 6 국내에서만 전기차가 오른건지. - 추가로 수행 / 

# 7 정책평가의 기준. - 지역별로 충전소가 골고루 퍼저있는가. 전기차의 대수만큼 충전소가 고르게 퍼저있는가. 인구비율로 따져 봤을때도 퍼저있는가. / (경제수준이 높은 곳이 더 많이 있는가.)
# https://m.etnews.com/20201117000196



In [9]:
!pip install chart_studio

     |████████████████████████████████| 64 kB 1.8 MB/s 


In [10]:
!pip install cufflinks

In [11]:
!pip install --upgrade plotly

     |████████████████████████████████| 23.9 MB 1.5 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [14]:
#필요한 라이브러리 불러오기
import numpy as np
import matplotlib as plt
import pandas as pd
import sys
import plotly.io as pio
pio.renderers.default = 'colab'
%matplotlib inline 
import numpy as np
import chart_studio
chart_studio.tools.set_credentials_file(username='username',api_key='api_key')
import plotly.graph_objects as go
# import plotly.graph_objects as go


In [15]:
#데이터 read 하기 

elcar = pd.read_excel('electric_car_data.xlsx', sheet_name=11,index_col=0,header=2,usecols="A,I") #필요한 정보만 가져오기 위해서 파라미터 지정
charger = pd.read_csv('charger_data.csv')
yecar = pd.read_excel('yearly_car.xlsx',index_col=0,header=[2,3])#필요한 정보만 가져오기 위해서 파라미터 지정
charger

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (13,20) have mixed types.Specify dtype option on import or set low_memory=False.



,STAT_NM,STAT_ID,CHGER_ID,CHGER_TYPE,ADDR,LAT,LON,USE_TIME,BUSI_ID,BUSI_NM,BUSI_CALL,CHGER_STAT,STAT_UPD_DT,POWER_TYPE,CITY_DO_CD,PARKING_FREE_YN,STAT_NOTE,LIMIT_YN,LIMIT_DETAIL,DEL_YN,DEL_DETAIL
0,종묘 공영주차장,ME000001,1,3,"서울특별시 종로구 종로 157, 지하주차장 4층 하층 T구역",37.571076,126.995880,24시간 이용가능,ME,환경부,1661-9408,9,2.021040e+13,급속(50kW),11,Y,NaN,N,NaN,N,NaN
1,세종로 공영주차장,ME000002,1,6,"서울특별시 종로구 세종대로 189, 지하주차장 4층 D구역 계단실 앞",37.573611,126.976011,24시간 이용가능,ME,환경부,1661-9408,2,2.021040e+13,급속(50kW),11,Y,NaN,N,NaN,N,NaN
2,그랜드앰배서더 서울,ME000003,1,6,"서울특별시 중구 동호로 287, 대형버스주차장",37.559352,127.002350,24시간 이용가능,ME,환경부,1661-9408,9,2.021040e+13,급속(50kW),11,N,NaN,N,NaN,N,NaN
3,한강진역 공영주차장,ME000004,1,3,"서울특별시 용산구 한남동 산10-84, 지상실외주차장",37.540085,127.002804,24시간 이용가능,ME,환경부,1661-9408,9,2.021040e+13,급속(50kW),11,Y,NaN,N,NaN,N,NaN
4,기아차 성동서비스센터,ME000005,1,3,"서울특별시 성동구 성수이로26길 61, 지상주차장 입구 좌측 경비실 옆",37.544834,127.064226,24시간 이용가능,ME,환경부,1661-9408,9,2.018070e+13,급속(50kW),11,N,20190626 공사로 인한 임시철거,Y,20190626 공사로 인한 임시철거,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69288,영남알프스복합웰컴센터,US200419,2,4,울산광역시 울주군 상북면 등억알프스리 517,35.556117,129.068415,24시간 이용가능,US,환경부,1661-9408,2,2.021040e+13,NaN,31,Y,NaN,N,NaN,N,NaN
69289,언양읍 행정복지센터,US200420,1,4,울산광역시 울주군 언양읍 동문길 7,35.569414,129.125781,24시간 이용가능,US,환경부,1661-9408,3,2.021040e+13,NaN,31,Y,NaN,N,NaN,N,NaN
69290,화봉공원 공영주차장,US200421,1,4,울산광역시 북구 화봉동 445-1,35.588995,129.364458,24시간 이용가능,US,환경부,1661-9408,2,2.021040e+13,NaN,31,Y,NaN,N,NaN,N,NaN
69291,화봉공원 공영주차장,US200421,2,4,울산광역시 북구 화봉동 445-1,35.588995,129.364458,24시간 이용가능,US,환경부,1661-9408,2,2.021040e+13,NaN,31,Y,NaN,N,NaN,N,NaN


In [16]:
# 오프라인 모드에서도 인터렉티브한 그래픽을 가능하도록 하기
# Enabling the offline mode for interactive plotting locally
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
#위 코드는 신경 안써도 됩니다.

yecar1=yecar.transpose() # 행열 변환
yecar2=pd.DataFrame(yecar1['합계']) # 시 군별 데이터는 필요없어서 전국데이터만 가져오기
yecar2.reset_index(inplace=True) # 년도를 없애지 않기위해 묶어두었던 멀티인덱스 해제
yecar2 # 결과 확인

,시도명,level_1,합계
0,2016.0,전기,10855
1,2016.0,수소,87
2,2016.0,하이브리드,233216
3,2016.0,친환경차,244158
4,2017.0,전기,25108
5,2017.0,수소,170
6,2017.0,하이브리드,313856
7,2017.0,친환경차,339134
8,2018.0,전기,55756
9,2018.0,수소,893


In [17]:
yecar3 = yecar2.copy() # yecar3 = yecar2.copy() 와 똑같은 뜻
yecar3['시도명'] = yecar3['시도명'].astype("int") # 실수형을 정수형으로 
yecar3['시도명'] = yecar3['시도명'].astype("category") # 정수형을 범주형으로 
yecar3['level_1'] = yecar3['level_1'].astype("category") # 범주형으로 변경
yecar3.rename(columns={'시도명':'년도', 'level_1':'차종', '합계':'등록대수'}, inplace=True) # 원래 데이터에 있던 컬럼이름을 원하는 데이터 출력에 맞게 수정
idx = yecar3[yecar3['차종']== '친환경차'].index
print(idx)
yecar3=yecar3.drop(idx)
yecar3.info(), yecar3.head() # 수정 데이터 확인
yecar3

Int64Index([3, 7, 11, 15, 19, 23], dtype='int64')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 22
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   년도      18 non-null     category
 1   차종      18 non-null     category
 2   등록대수    18 non-null     int64   
dtypes: category(2), int64(1)
memory usage: 724.0 bytes


,년도,차종,등록대수
0,2016,전기,10855
1,2016,수소,87
2,2016,하이브리드,233216
4,2017,전기,25108
5,2017,수소,170
6,2017,하이브리드,313856
8,2018,전기,55756
9,2018,수소,893
10,2018,하이브리드,405084
12,2019,전기,89918


In [18]:
yecar3.head()# 수정 데이터 확인

,년도,차종,등록대수
0,2016,전기,10855
1,2016,수소,87
2,2016,하이브리드,233216
4,2017,전기,25108
5,2017,수소,170


In [19]:
import plotly.express as px

In [20]:
fig = px.bar(yecar3, x='년도', y='등록대수', color='차종') # x축을 연도로 y축을 등록대수로 색갈은 차종으로 구별
pio.renderers.default = 'colab' 
fig #확인

In [40]:
fig = px.bar(yecar3[yecar3.차종!='하이브리드'], x='년도', y='등록대수', color='차종') # x축을 연도로 y축을 등록대수로 색갈은 차종으로 구별
pio.renderers.default = 'colab' 
fig #확인

In [21]:
fig = px.bar(yecar3[yecar3.차종=='전기'], x='년도', y='등록대수', color='차종') # x축을 연도로 y축을 등록대수로 색갈은 차종으로 구별
pio.renderers.default = 'colab' 
fig #확인

In [22]:
yecar3.index

Int64Index([0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 14, 16, 17, 18, 20, 21, 22], dtype='int64')

In [23]:
elcar

,전기차
시도,NaN
서울,29062.0
부산,8838.0
대구,12752.0
인천,8664.0
광주,4573.0
대전,5969.0
울산,4503.0
세종,1540.0
경기,29984.0


In [24]:
charger.head()

,STAT_NM,STAT_ID,CHGER_ID,CHGER_TYPE,ADDR,LAT,LON,USE_TIME,BUSI_ID,BUSI_NM,BUSI_CALL,CHGER_STAT,STAT_UPD_DT,POWER_TYPE,CITY_DO_CD,PARKING_FREE_YN,STAT_NOTE,LIMIT_YN,LIMIT_DETAIL,DEL_YN,DEL_DETAIL
0,종묘 공영주차장,ME000001,1,3,"서울특별시 종로구 종로 157, 지하주차장 4층 하층 T구역",37.571076,126.995880,24시간 이용가능,ME,환경부,1661-9408,9,2.021040e+13,급속(50kW),11,Y,NaN,N,NaN,N,NaN
1,세종로 공영주차장,ME000002,1,6,"서울특별시 종로구 세종대로 189, 지하주차장 4층 D구역 계단실 앞",37.573611,126.976011,24시간 이용가능,ME,환경부,1661-9408,2,2.021040e+13,급속(50kW),11,Y,NaN,N,NaN,N,NaN
2,그랜드앰배서더 서울,ME000003,1,6,"서울특별시 중구 동호로 287, 대형버스주차장",37.559352,127.002350,24시간 이용가능,ME,환경부,1661-9408,9,2.021040e+13,급속(50kW),11,N,NaN,N,NaN,N,NaN
3,한강진역 공영주차장,ME000004,1,3,"서울특별시 용산구 한남동 산10-84, 지상실외주차장",37.540085,127.002804,24시간 이용가능,ME,환경부,1661-9408,9,2.021040e+13,급속(50kW),11,Y,NaN,N,NaN,N,NaN
4,기아차 성동서비스센터,ME000005,1,3,"서울특별시 성동구 성수이로26길 61, 지상주차장 입구 좌측 경비실 옆",37.544834,127.064226,24시간 이용가능,ME,환경부,1661-9408,9,2.018070e+13,급속(50kW),11,N,20190626 공사로 인한 임시철거,Y,20190626 공사로 인한 임시철거,N,NaN


In [25]:
charger.groupby(['CITY_DO_CD']).count() 
idx_nm_1 = charger[charger['LON'] == 124].index # 위도 경도 33 124인 결측치 제거를 위한 인덱스 저장


charger.dropna(subset=['LAT'],inplace=True) #위도 경도 결측치제거
charger.dropna(subset=['LON'],inplace=True) #위도 경도 결측치제거



charger = charger.drop(idx_nm_1) # 인덱스 정보로 위도 경도 33 124인 결측치 제거
charger.isnull().sum() #결측치 확인

STAT_NM                0
STAT_ID                0
CHGER_ID               0
CHGER_TYPE             0
ADDR                  95
LAT                    0
LON                    0
USE_TIME            7816
BUSI_ID                0
BUSI_NM                0
BUSI_CALL           4984
CHGER_STAT             0
STAT_UPD_DT         1594
POWER_TYPE         64572
CITY_DO_CD             0
PARKING_FREE_YN      606
STAT_NOTE          59255
LIMIT_YN               0
LIMIT_DETAIL       36714
DEL_YN                 0
DEL_DETAIL         68945
dtype: int64

In [26]:
elcar.dropna(inplace=True)

In [27]:
elcar.rename(index={'합계':'전국', # 서로 다른 데이터에서 지역이름을 통일하기 위해 지역이름을 행정코드로 변경
'서울': 11,
'부산':	26,
'대구':	27,
'인천':	28,
'광주':	29,
'대전':	30,
'울산':	31,
'세종':	36,
'경기':	41,
'강원':	42,
'충북':	43,
'충남':	44,
'전북':	45,
'전남':	46,
'경북':	47,
'경남':	48,
'제주':	50

}, inplace=True)
type(elcar)
elcar.astype("int") # 정수형으로로 변경

,전기차
11,29062
26,8838
27,12752
28,8664
29,4573
30,5969
31,4503
36,1540
41,29984
42,5930


In [28]:
charger_count = charger.groupby(['CITY_DO_CD'])['STAT_NM'].count() # 충전기 개수를 세서 charger_count에 저장
charger_count = pd.DataFrame(charger_count) # charger_count를 데이터 프레임으로 형변환
# charger_count.rename(columns={ "CITY_DO_CD":"지역"} ) 신경 안쓰셔도 됩니다.
charger_count.astype("int") # charger_count를 정수형으로 변경

,STAT_NM
CITY_DO_CD,
11,8880
26,3669
27,4812
28,2424
29,2316
30,1877
31,1257
36,740
41,16801


In [29]:
slow_chagers = charger[charger['POWER_TYPE'].isnull()].groupby(['CITY_DO_CD'])['STAT_NM'].count() # 완속충전기 개수 세서 slow_chagers에 저장

In [30]:
slow_chagers # 결과 확인 

CITY_DO_CD
11     8607
26     3582
27     4677
28     2293
29     2233
30     1782
31     1140
36      706
41    16244
42     2013
43     2110
44     2886
45     2182
46     2358
47     3712
48     3838
50     4209
Name: STAT_NM, dtype: int64

In [31]:
result = []
sum = 0
for i in range( len(list( elcar['전기차']))-1 ): # 전기차 -1 만큼 반복 (전국데이터는 제외해야 하므로)

  result.append( ( ( charger_count['STAT_NM'].iloc[i]- (slow_chagers.iloc[i]*0.2)) / elcar['전기차'].iloc[i]) *100 ) # (reslut에 ( 충전기대수 - (완속충전기*0.2) / 전기차대수 ) * 100 삽입) 보기좋게 하기위해 100을 곱함

  sum+= ( ( charger_count['STAT_NM'].iloc[i]- (slow_chagers.iloc[i]*0.2)) / elcar['전기차'].iloc[i])*100 # reslut에 넣은값 그대로 sum 더해줌 (전국 평균 구하기 위해서)

result.append(sum/17) # sum에 저장한 값을 지역 개수인 17로 나눠줘서 평균값 도출 후 resulut에 삽입



result #결과 확인


[24.632165714678962,
 33.40801086218602,
 30.399937264742782,
 22.68467220683287,
 40.87907281871856,
 25.474953928631262,
 22.85143237841439,
 38.883116883116884,
 45.19810565635006,
 34.323777403035415,
 36.45429362880886,
 37.999719848718314,
 42.780991735537185,
 35.696883404524606,
 50.27027027027027,
 38.64329093617509,
 18.287298387096772,
 34.05105843104931]

In [32]:
df1 = elcar # 똑같은 데이터 프레임 만들어줌

In [33]:
df1['전기차'] = result  # 계산했던 지수값 넣어줌
df1 #결과 확인

,전기차
11,24.632166
26,33.408011
27,30.399937
28,22.684672
29,40.879073
30,25.474954
31,22.851432
36,38.883117
41,45.198106
42,34.323777


In [34]:
df1.astype("int") # 실수형을 보기좋게 정수형으로 / 지역별 전기차 적합 '지수'             

,전기차
11,24
26,33
27,30
28,22
29,40
30,25
31,22
36,38
41,45
42,34


In [35]:
df1.rename(index={'합계':'전국'} , inplace=True) 
df1 = df1.astype("int")

In [36]:
df1

,전기차
11,24
26,33
27,30
28,22
29,40
30,25
31,22
36,38
41,45
42,34


In [37]:
df1.to_csv("local_index.csv",encoding='cp949')